In [3]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/59/1f/4975d1ab3ed2244053876321ef65bc02935daed67da76c6e7d65900772a3/torch-2.2.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Obtaining dependency information for mpmath>=0.19 from https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB 991.0 kB/s eta 0:03:21
   ------------------

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
from deep_translator import GoogleTranslator
from nanoid import generate
from slugify import slugify
from transformers import pipeline

c:\Users\JIME\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### URL: https://www.epam.com/careers/job-listings?country=Mexico

In [ ]:
response = requests.get('https://www.epam.com/services/vacancy/search?locale=en&limit=200&recruitingUrl=%2Fcontent%2Fepam%2Fen%2Fcareers%2Fjob-listings%2Fjob&query=&country=Mexico&sort=relevance&offset=0&searchType=placeOfWorkFilter&_=1709566523032')

dictionario = response.json()

print(dictionario)

print(len(dictionario['result']))

### Ejercicio para extraer el contenido dentro de un div con etiquetas y crear un diccionario 

In [ ]:
def extraer_contenido_div(url):
    # Realizamos la solicitud GET a la URL
    response = requests.get(url)
    
    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        # Parseamos el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontramos el div con la clase deseada
        div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')  
        
        # Verificamos si se encontró el div
        if div_contenido:
            # Inicializamos una lista para almacenar el contenido
            contenido = []
            
            # Recorremos todos los elementos <h3> dentro del div
            for h3_tag in div_contenido.find_all('h3'):
                # Agregamos el texto de cada título a la lista de contenido
                contenido.append(h3_tag.text.strip())
            
            return contenido
        else:
            print("No se encontró el div con la clase especificada.")
    else:
        print("Error al obtener la página:", response.status_code)

In [ ]:
# URL de la página a scrapear (link extraido del primer scrap)
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'

# Llamamos a la función y mostramos el contenido extraído
contenido_div = extraer_contenido_div(url)
if contenido_div:
    print("Contenido del div:")
    for item in contenido_div:
        print(item)

In [ ]:
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'
# Parseamos el contenido HTML
soup = BeautifulSoup(url, 'html.parser')

location=soup.find("span", class_="vacancy-details-23__location").text.strip()
title = soup.find("h1", class_="vacancy-details-23__job-title").text.strip()
# Encontramos el div con la clase "vacancy-details-23__content-holder"
div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')

# Verificamos si se encontró el div
if div_contenido:
    # Inicializamos un diccionario para almacenar el contenido
    contenido_por_titulo = {}
    
    # Recorremos todos los elementos <h3> dentro del div
    for h3_tag in div_contenido.find_all('h3'):
        # Inicializamos una lista para almacenar los elementos de lista correspondientes a este título
        elementos_li = []
        
        # Buscamos el siguiente elemento <ul> hermano del título <h3>
        ul_tag = h3_tag.find_next_sibling('ul')
        
        # Recorremos todos los elementos <li> dentro del <ul>
        for li_tag in ul_tag.find_all('li'):
            # Agregamos el texto del elemento <li> a la lista de elementos_li
            elementos_li.append(li_tag.text.strip())
        
        # Almacenamos los elementos de lista en el diccionario bajo el título correspondiente
        contenido_por_titulo[h3_tag.text.strip()] = elementos_li

# Mostramos el contenido almacenado en variables con los nombres de los títulos
for titulo, elementos in contenido_por_titulo.items():
    # Creamos una variable con el nombre del título y le asignamos la lista de elementos
    globals()[titulo.replace(" ", "_").lower()] = elementos

# Ahora puedes acceder a cada lista de elementos utilizando el nombre de la variable correspondiente al título
print("Contenido almacenado en variables:")
print("Responsibilities:", responsibilities)
print("Requirements:", requirements)
print("Technologies:", technologies)
print("We_Offer:", we_offer)
print("Conditions:", conditions)

In [ ]:
def extraer_contenido_h3_ul_desde_url(url):
    try:
        # Realizamos la solicitud GET a la URL
        response = requests.get(url)
        
        # Verificamos si la solicitud fue exitosa
        if response.status_code == 200:
            # Parseamos el contenido HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            location=soup.find("span", class_="vacancy-details-23__location").text.strip()
            title = soup.find("h1", class_="vacancy-details-23__job-title").text.strip()
            # Encontramos el div con la clase "vacancy-details-23__content-holder"
            div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')
            
            # Verificamos si se encontró el div
            if div_contenido:
                # Inicializamos un diccionario para almacenar el contenido
                contenido_por_titulo = {}
                
                # Recorremos todos los elementos <h3> dentro del div
                for h3_tag in div_contenido.find_all('h3'):
                    # Inicializamos una lista para almacenar los elementos de lista correspondientes a este título
                    elementos_li = []
                    
                    # Buscamos el siguiente elemento <ul> hermano del título <h3>
                    ul_tag = h3_tag.find_next_sibling('ul')
                    
                    # Recorremos todos los elementos <li> dentro del <ul>
                    for li_tag in ul_tag.find_all('li'):
                        # Agregamos el texto del elemento <li> a la lista de elementos_li
                        elementos_li.append(li_tag.text.strip())
                    
                    # Almacenamos los elementos de lista en el diccionario bajo el título correspondiente
                    contenido_por_titulo[h3_tag.text.strip()] = elementos_li
                
                return contenido_por_titulo
            else:
                print("No se encontró el div con la clase especificada.")
        else:
            print("Error al obtener la página:", response.status_code)
    except Exception as e:
        print("Ocurrió un error:", e)

In [ ]:
# Ejemplo de uso
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'
contenido_por_titulo = extraer_contenido_h3_ul_desde_url(url)

if contenido_por_titulo:
    # Mostramos el contenido almacenado en el diccionario
    for titulo, elementos in contenido_por_titulo.items():
        print(f"{titulo}:")
        for elemento in elementos:
            print(f"- {elemento}")
        print()

### Scrapeo de la url a partir del json obtenido de la api

In [18]:
url_api= "https://www.epam.com/services/vacancy/search?locale=en&limit=200&recruitingUrl=%2Fcontent%2Fepam%2Fen%2Fcareers%2Fjob-listings%2Fjob&query=&country=Mexico&sort=relevance&offset=0&searchType=placeOfWorkFilter&_=1709566523032"

url="https://www.epam.com/careers/job-listings?country=Mexico"


In [22]:
def obtener_enlaces_vacantes(url):
    # Realizar la solicitud GET a la API y obtener la respuesta JSON
    response = requests.get(url)
    data = response.json()
    
    # Verificar si la solicitud fue exitosa y si se obtuvieron datos
    if response.status_code == 200 and data:
        # Extraer los enlaces de las vacantes de empleo del JSON
        vacancy_links = [item["url"] for item in data["result"]]
        
        # Crear un DataFrame a partir de los enlaces obtenidos
        df = pd.DataFrame({"Enlace": vacancy_links})
        return df
    else:
        print("No se pudo acceder a la API de vacantes de empleo o no se obtuvieron datos")
        return None

# URL de la API que devuelve los datos JSON de las vacantes de empleo
url_api = "https://www.epam.com/services/vacancy/search?locale=en&limit=200&recruitingUrl=%2Fcontent%2Fepam%2Fen%2Fcareers%2Fjob-listings%2Fjob&query=&country=Mexico&sort=relevance&offset=0&searchType=placeOfWorkFilter&_=1709566523032"

# Llamar a la función para obtener los enlaces de las vacantes de empleo
df_enlaces_vacantes = obtener_enlaces_vacantes(url_api)

In [23]:
df_enlaces_vacantes

,Enlace
0,https://www.epam.com/careers/job-listings/job....
1,https://www.epam.com/careers/job-listings/job....
2,https://www.epam.com/careers/job-listings/job....
3,https://www.epam.com/careers/job-listings/job....
4,https://www.epam.com/careers/job-listings/job....
...,...
100,https://www.epam.com/careers/job-listings/job....
101,https://www.epam.com/careers/job-listings/job....
102,https://www.epam.com/careers/job-listings/job....
103,https://www.epam.com/careers/job-listings/job....


In [25]:
df_enlaces_vacantes["Enlace"][1]

'https://www.epam.com/careers/job-listings/job.87801.big-data-support-engineer_guadalajara_mexico'

### Aqúi realizo el srcaping de cada enlace de vacante


In [27]:

def get_job_details(df_enlaces_vacantes):
    # Lista para almacenar los detalles de cada vacante
    detalles_vacantes = []
    
    # Iterar sobre cada enlace de la columna "Enlace" del DataFrame
    for enlace in df_enlaces_vacantes["Enlace"]:
        # Realizar la solicitud GET al enlace de la vacante
        response = requests.get(enlace)
        
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Parsear el HTML de la página con BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extraer los detalles de la vacante
            location=soup.find("div", class_="location").text.strip()
            title = soup.find("h1", class_="title").text.strip()
            responsabilities = soup.find("div", class_="responsibilities").text.strip()
            requirements = soup.find("div", class_="requirements").text.strip()
            technologies = soup.find("div", class_="technologies").text.strip()
            we_offer = soup.find("div", class_="we-offer").text.strip()
            
            # Agregar los detalles a la lista
            detalles_vacantes.append({
                "Location": location
                "Title": title,
                "Responsibilities": responsabilities,
                "Requirements": requirements,
                "Technologies": technologies,
                "We_Offer": we_offer,
                "URL": enlace
            })
        else:
            print(f"No se pudo acceder a la vacante: {enlace}")
    
    # Convertir la lista de detalles de vacantes a un DataFrame
    df_detalles_vacantes = pd.DataFrame(detalles_vacantes)
    return df_detalles_vacantes

# Llamar a la función para obtener los enlaces de las vacantes de empleo
df_enlaces_vacantes = obtener_enlaces_vacantes(url_api)

# Verificar si se obtuvieron los enlaces correctamente
if df_enlaces_vacantes is not None:
    # Llamar a la función para obtener los detalles de las vacantes de empleo
    df_detalles_vacantes = get_job_details(df_enlaces_vacantes)
    
    # Imprimir el DataFrame con los detalles de las vacantes de empleo
    print("DataFrame con los detalles de las vacantes de empleo:")
    print(df_detalles_vacantes)
else:
    print("No se pudieron obtener los enlaces de las vacantes de empleo")


SyntaxError: invalid syntax (2883986245.py, line 26)

In [ ]:
def epam(df: pd.DataFrame) -> pd.DataFrame:
    df["name"] = clean_title(df["name"])
    df["companyName"] = df["companyName"].replace(df["companyName"].values, "Apple")
    df["Otros_requisitos"] = df["Otros_requisitos"].fillna("")
    df["Educación_Experiencia"] = df["Educación_Experiencia"].fillna("")
    columns_to_merge = [
        "Responsabilities",
        "Requirements",
        "We Offer",
        "conditions"
    ]
    column_merge_new = "Req_txt"
    columns_to_delete = [
        "Responsabilities",
        "Requirements",
        "conditions"
    ]
    df = merge_clean_delete(df, column_merge_new, columns_to_merge, columns_to_delete)
    df["skills"] = df["Req_txt"].apply(find_skills)
    df["skills"] = df["skills"].drop_duplicates(keep="last")
    df["skills"] = df["skills"].apply(dict_skills)
    df["tools"] = df["Req_txt"].apply(find_tools)
    df["tools"] = df["tools"].apply(dict_tools)
    df["tools"] = df["tools"].drop_duplicates(keep="last")
    df["aptitudes"] = df["Req_txt"].apply(find_aptitudes)
    df["aptitudes"] = df["aptitudes"].drop_duplicates(keep="last")
    df["aptitudes"] = df["aptitudes"].apply(dict_aptitudes)
    df["languages"] = df["Req_txt"].apply(find_languages)
    df["languages"] = df["languages"].drop_duplicates(keep="last")
    df["languages"] = df["languages"].apply(dict_languages)
    df = df.drop("Req_txt", axis=1)
    df = translate_columns_deep_translator(
        df, column_names=["aptitudes", "languages", "skills"]
    )
    df = complete_dataframe(df)
    df = fill_data(df, companyId="epam")
    df["skills"] = df["skills"].apply(json_nan)
    df["tools"] = df["tools"].apply(json_nan)
    df["aptitudes"] = df["aptitudes"].apply(json_nan)
    df["languages"] = df["languages"].apply(json_nan)
    return df

In [ ]:
diccionario_epam = {
    "userId": "epam",
    "name": "epam",
    "description": """
                    EPAM Systems, Inc. es una empresa estadounidense que se especializa en servicios de ingeniería de software, ingeniería de plataformas digitales y diseño de productos digitales, que opera en Newtown, Pensilvania. EPAM es miembro fundador de la Alianza MACH.
                    """,
    "website": "www.epam.com//",
    "sector": "Tecnologías de la información y la comunicación",
    "isExternal": True,
}

df_epam = pd.DataFrame(diccionario_epam, index=[0])